In [157]:
from solution_library import *

import numpy as np, matplotlib.pyplot as plt, pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from sklearn.preprocessing import MinMaxScaler

import warnings

torch.set_default_dtype(torch.float64)

In [158]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(29, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

In [159]:
with warnings.catch_warnings(action="ignore"):
    x_df, y_df = claims_preprocess_get_xy()

In [160]:
x = x_df.values.astype(np.float64)
y = y_df.values.astype(np.float64)

scalar = MinMaxScaler() # USE THIS SAME SCALAR LATER
x_norm = scalar.fit_transform(x)

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x_norm, y, test_size = 0.15, random_state=0)

inputs = torch.from_numpy(x_train)
targets = torch.from_numpy(y_train)
x_test = torch.from_numpy(x_test)
y_test = torch.from_numpy(y_test)

In [166]:
x_norm

array([[0.        , 0.        , 0.        , ..., 0.23788989, 0.50900548,
        0.05475306],
       [0.        , 0.        , 1.        , ..., 0.12226626, 0.34651527,
        0.50502132],
       [0.        , 1.        , 0.        , ..., 0.00970999, 0.        ,
        0.93960655],
       ...,
       [0.        , 0.        , 1.        , ..., 0.20338207, 0.55755677,
        0.36112258],
       [0.        , 0.        , 0.        , ..., 0.039806  , 0.32419734,
        0.32824322],
       [0.        , 0.        , 0.        , ..., 0.28838187, 0.64996085,
        0.42014032]])

In [161]:
model = Net()

criterion = nn.MSELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.02)
# optimizer = optim.SGD(model.parameters(), lr=0.025)
optimizer = optim.LBFGS(model.parameters(), lr = 0.01)


In [162]:
'''# FOR ADAM

batch_size = 64
dataset = torch.utils.data.TensorDataset(inputs, targets)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

count = 0

for epoch in range(150):
    try:
        for inputs_batch, targets_batch in dataloader:
            optimizer.zero_grad()  # zero the parameter gradients
            outputs = model(inputs_batch)  # forward pass
            loss = torch.sqrt(criterion(outputs, targets_batch))  # compute loss
            loss.backward()  # backward pass
            optimizer.step()  # update weights

        # Evaluate on the validation set
        with torch.no_grad():
            val_outputs = model(x_test)
            val_loss = torch.sqrt(criterion(val_outputs, y_test))

        print(f'Epoch: {epoch + 1:4d}. Loss: {loss.item():.3f}. Validation Loss: {val_loss.item():.3f}')

        if loss.item() < 700 and count > 5:
            break
        elif loss.item() < 700:
            count += 1
            pass

    except KeyboardInterrupt:
        print('Ended Training Early')
        break
print('Finished Training')'''

"# FOR ADAM\n\nbatch_size = 64\ndataset = torch.utils.data.TensorDataset(inputs, targets)\ndataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)\n\ncount = 0\n\nfor epoch in range(150):\n    try:\n        for inputs_batch, targets_batch in dataloader:\n            optimizer.zero_grad()  # zero the parameter gradients\n            outputs = model(inputs_batch)  # forward pass\n            loss = torch.sqrt(criterion(outputs, targets_batch))  # compute loss\n            loss.backward()  # backward pass\n            optimizer.step()  # update weights\n\n        # Evaluate on the validation set\n        with torch.no_grad():\n            val_outputs = model(x_test)\n            val_loss = torch.sqrt(criterion(val_outputs, y_test))\n\n        print(f'Epoch: {epoch + 1:4d}. Loss: {loss.item():.3f}. Validation Loss: {val_loss.item():.3f}')\n\n        if loss.item() < 700 and count > 5:\n            break\n        elif loss.item() < 700:\n            count += 1

In [163]:
model1 = Net()

criterion1 = nn.MSELoss()
optimizer1 = optim.LBFGS(model1.parameters(), lr = 0.01)


In [164]:
batch_size = 64
dataset = torch.utils.data.TensorDataset(inputs, targets)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

count = 0

for epoch in range(2000):
    try:
        for inputs_batch, targets_batch in dataloader:
            
            def closure():
                optimizer1.zero_grad()  # zero the parameter gradients
                outputs = model(inputs_batch)  # forward pass
                loss = torch.sqrt(criterion1(outputs, targets_batch))  # compute loss
                loss.backward()  # backward pass
                return loss
                
            optimizer.step(closure)  # update weights

        # Evaluate on the validation set
        with torch.no_grad():
            val_outputs = model1(x_test)
            val_loss = torch.sqrt(criterion1(val_outputs, y_test))

        print(f'Epoch: {epoch + 1:4d}. Loss: {loss.item():.3f}. Validation Loss: {val_loss.item():.3f}')

        if loss.item() < 700 and count > 5:
            break
        elif loss.item() < 700:
            count += 1
            pass

    except KeyboardInterrupt:
        print('Ended Training Early')
        break
print('Finished Training')

c:\Users\kyle\OneDrive\Programs\Cooperators_2024\.venv\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\kyle\OneDrive\Programs\Cooperators_2024\.venv\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([15])) that is different to the input size (torch.Size([15, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\kyle\OneDrive\Programs\Cooperators_2024\.venv\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([2126])) that is different to the input size (torch.Size([2126, 1])). This will likely lead to i

Epoch:    1. Loss: 1586.872. Validation Loss: 1815.819
Epoch:    2. Loss: 1586.872. Validation Loss: 1815.819
Epoch:    3. Loss: 1586.872. Validation Loss: 1815.819
Epoch:    4. Loss: 1586.872. Validation Loss: 1815.819
Epoch:    5. Loss: 1586.872. Validation Loss: 1815.819
Epoch:    6. Loss: 1586.872. Validation Loss: 1815.819
Epoch:    7. Loss: 1586.872. Validation Loss: 1815.819
Epoch:    8. Loss: 1586.872. Validation Loss: 1815.819
Epoch:    9. Loss: 1586.872. Validation Loss: 1815.819
Epoch:   10. Loss: 1586.872. Validation Loss: 1815.819
Epoch:   11. Loss: 1586.872. Validation Loss: 1815.819
Epoch:   12. Loss: 1586.872. Validation Loss: 1815.819
Epoch:   13. Loss: 1586.872. Validation Loss: 1815.819
Epoch:   14. Loss: 1586.872. Validation Loss: 1815.819
Epoch:   15. Loss: 1586.872. Validation Loss: 1815.819
Epoch:   16. Loss: 1586.872. Validation Loss: 1815.819
Epoch:   17. Loss: 1586.872. Validation Loss: 1815.819
Epoch:   18. Loss: 1586.872. Validation Loss: 1815.819
Epoch:   1

In [ ]:
model2 = Net()

criterion2 = nn.MSELoss()
optimizer2 = optim.LBFGS(model2.parameters(), lr = 0.01)


In [ ]:
for epoch in range(10000):
    try:
        def closure():
            optimizer2.zero_grad()  # zero the parameter gradients
            outputs = model2(inputs_batch)  # forward pass
            loss = torch.sqrt(criterion2(outputs, targets_batch))  # compute loss
            loss.backward()  # backward pass
            return loss
            
        optimizer2.step(closure)  # update weights

        # Evaluate on the validation set
        with torch.no_grad():
            val_outputs = model2(x_test)
            val_loss = torch.sqrt(criterion2(val_outputs, y_test))

        print(f'Epoch: {epoch + 1:4d}. Loss: {loss.item():.3f}. Validation Loss: {val_loss.item():.3f}')



    except KeyboardInterrupt:
        print('Ended Training Early')
        break
print('Finished Training')

c:\Users\kyle\OneDrive\Programs\Cooperators_2024\.venv\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\kyle\OneDrive\Programs\Cooperators_2024\.venv\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([2126])) that is different to the input size (torch.Size([2126, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch:    1. Loss: 1586.872. Validation Loss: 2080.524
Epoch:    2. Loss: 1586.872. Validation Loss: 1654.992
Epoch:    3. Loss: 1586.872. Validation Loss: 1536.082
Epoch:    4. Loss: 1586.872. Validation Loss: 1486.061
Epoch:    5. Loss: 1586.872. Validation Loss: 1460.129
Epoch:    6. Loss: 1586.872. Validation Loss: 1444.363
Epoch:    7. Loss: 1586.872. Validation Loss: 1434.659
Epoch:    8. Loss: 1586.872. Validation Loss: 1428.873
Epoch:    9. Loss: 1586.872. Validation Loss: 1425.349
Epoch:   10. Loss: 1586.872. Validation Loss: 1423.282
Epoch:   11. Loss: 1586.872. Validation Loss: 1422.037
Epoch:   12. Loss: 1586.872. Validation Loss: 1421.268
Epoch:   13. Loss: 1586.872. Validation Loss: 1420.692
Epoch:   14. Loss: 1586.872. Validation Loss: 1420.206
Epoch:   15. Loss: 1586.872. Validation Loss: 1419.774
Ended Training Early
Finished Training


In [ ]:
# for epoch in range(9000):
#     try:
#         outputs = model(inputs) # forward + backward + optimize        
#         loss = criterion(outputs, targets)

#         optimizer.zero_grad() # zero the parameter gradients
#         loss.backward()
#         optimizer.step()

#         print(f'Epoch: {epoch + 1:4d}. Loss: {loss:.3f}')

#         if loss < 1E-7:
#             break
#             # pass

#     except KeyboardInterrupt:
#         print('Ended Training Early')
#         break


# print('Finished Training')

In [ ]:
x_new, ids = preprocess_x(load_tests())

x_new = x_new.values.astype(np.float64)

x_new_norm = scalar.transform(x_new)

inputs_pred = torch.from_numpy(x_new_norm)

model.eval() # convert to evaluation mode

with torch.no_grad(): # forward pass
    predictions = model(inputs_pred)



c:\Users\kyle\OneDrive\Programs\Cooperators_2024\solution_library.py:119: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  claims['pol_pay_freq'] = claims['pol_pay_freq'].replace( {'Biannual': 2, 'Yearly': 1, 'Monthly': 12, 'Quarterly': 4} )
c:\Users\kyle\OneDrive\Programs\Cooperators_2024\solution_library.py:120: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  claims['pol_payd'] = claims['pol_payd'].replace( {'No': 0, 'Yes': 1} )
c:\Users\kyle\OneDrive\Programs\Cooperators_2024\solution_library.py:121: FutureWarning: Downcasting behav

In [ ]:
rmse = RMSE(predictions, y_test)
rmse

tensor(1416.9009)

In [ ]:
predictions = pd.DataFrame(predictions, columns=['claim_amount'])

save_data = pd.concat([ids, predictions], axis = 1)

save_data.to_csv(f'predictions/pred_nn_v2.csv')

predictions

,claim_amount
0,1132.507558
1,1122.830905
2,1143.387248
3,1126.137521
4,1136.761373
...,...
4135,1131.069713
4136,1117.706367
4137,1134.945978
4138,1132.106309


In [165]:
# Predictions for model1

x_new, ids = preprocess_x(load_tests())

x_new = x_new.values.astype(np.float64)

x_new_norm = scalar.transform(x_new)

inputs_pred = torch.from_numpy(x_new_norm)

model1.eval() # convert to evaluation mode

with torch.no_grad(): # forward pass
    predictions = model1(inputs_pred)

predictions = pd.DataFrame(predictions, columns=['claim_amount'])

save_data = pd.concat([ids, predictions], axis = 1)

save_data.to_csv(f'predictions/pred_nn_v_3.csv')

predictions

c:\Users\kyle\OneDrive\Programs\Cooperators_2024\solution_library.py:119: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  claims['pol_pay_freq'] = claims['pol_pay_freq'].replace( {'Biannual': 2, 'Yearly': 1, 'Monthly': 12, 'Quarterly': 4} )
c:\Users\kyle\OneDrive\Programs\Cooperators_2024\solution_library.py:120: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  claims['pol_payd'] = claims['pol_payd'].replace( {'No': 0, 'Yes': 1} )
c:\Users\kyle\OneDrive\Programs\Cooperators_2024\solution_library.py:121: FutureWarning: Downcasting behav

,claim_amount
0,0.078086
1,0.084223
2,0.022502
3,0.078890
4,0.033416
...,...
4135,0.023436
4136,0.091254
4137,0.070797
4138,0.043806


In [ ]:
# Predictions for model2

x_new, ids = preprocess_x(load_tests())

x_new = x_new.values.astype(np.float64)

x_new_norm = scalar.transform(x_new)

inputs_pred = torch.from_numpy(x_new_norm)

model2.eval() # convert to evaluation mode

with torch.no_grad(): # forward pass
    predictions = model2(inputs_pred)

predictions = pd.DataFrame(predictions, columns=['claim_amount'])

save_data = pd.concat([ids, predictions], axis = 1)

save_data.to_csv(f'predictions/pred_nn_v_4.csv')

predictions